In [4]:
import pandas as pd

In [1]:
import json

In [34]:
from preprocess.preprocess import preprocess_text

In [36]:
from tqdm import tqdm_notebook

In [9]:
translations = json.load(open("./result-toxic.json"))

In [5]:
perspective_df = pd.read_csv("./train.csv")

In [6]:
toxic_classes = ['threat', 'insult', 'identity_hate', 'toxic']

In [7]:
perspective_df["real_toxic"] = (perspective_df[toxic_classes].sum(axis=1) > 0).astype(int)

In [12]:
translations

{'b2ce8a88d87caf5f': 'попробуйте android и dreamguy оба очень непопулярны с другими викианами, и оба кажутся дырами необразованными высокомерными и просто тупыми, как пост beckjord',
 '670710bed58c2532': 'eh wha это предложение находится чуть выше другого раздела коррозия металлических коробок бенто кислотой умебоши была популярна в Японии и в конечном итоге делает отверстие в середине крышки может кто-то распутать его я бы, но я не знаю, что он пытается сказать',
 '6a5b05645cbff2bd': 'Привет, я загрузил лицензию Арнольда Мюррея в Википедию и обновил ссылки, однако я не уверен, что я сделал авторское право правильно категории авторских прав кажутся довольно странными лицензия, как отмечено, из офиса Бентон Каунти АР клерк s и никаких авторских прав не применяется это публичная запись вы можете проверить его и убедиться, что это правильно Спасибо',
 'dcffbb62d3ebdf69': 'это был не вандализм для вас, возможно, но для этого пользователя нет данных о ваших личных атаках',
 'f5d4784b378dfa9

In [13]:
perspective_df["translate"] = perspective_df["id"].apply(lambda x: translations.get(x))

In [21]:
perspective_df["translate"] = perspective_df["translate"].fillna("API Error")

In [24]:
perspective_df.drop(perspective_df[perspective_df["translate"].str.startswith("API Error")].index, inplace=True)

In [32]:
perspective_df["toxic_class"] = (perspective_df[toxic_classes].sum(axis=1) > 0).astype(int)

In [37]:
perspective_df["lemmatize"] = [preprocess_text(text) for text in tqdm_notebook(perspective_df["translate"])]

In [53]:
with open('data/toxicity.pos', "w") as file:
    for comment in perspective_df[perspective_df["toxic_class"] == 1]["lemmatize"]:
        file.write(comment + '\n')

In [56]:
with open('data/toxicity.neg', "w") as file:
    for comment in perspective_df[perspective_df["toxic_class"] == 0]["lemmatize"]:
        file.write(comment + '\n')